# Training Notebook

In [1]:
import glob
import os

import cv2
import xml.etree.ElementTree as ET

import inference_utils

In [2]:
class FishCounter(object):
    def __init__(self,model_config,model_weights):
        self.net = cv2.dnn.readNetFromDarknet(model_config, model_weights)
        self.input_width = 416
        self.input_height = 416

    def inference(self,frame):
        """Run inference on single frame."""
        # Create a 4D blob from a frame.
        blob = cv2.dnn.blobFromImage(
            frame, 1 / 255, (self.input_width, self.input_height), [0, 0, 0], 1, crop=False
        )

        # Sets the input to the network
        self.net.setInput(blob)

        # Runs the forward pass to get output of the output layers
        outs = self.net.forward(inference_utils.get_outputs_names(net=self.net))

    def annotated_frame(self):
        """Returns main frame with fish highlighted."""
        frame = self.frame.copy()

In [3]:
fullname = os.path.join(path, filename)
tree = ET.parse(fullname)
root = tree.getroot()

NameError: name 'path' is not defined

In [7]:
DATA_DIR = os.path.join("..","data","Herring")

TRAINING_DIR = os.path.join(DATA_DIR,"Training","IRWA 2017 (Large+annotations)")
TESTING_DIR = os.path.join(DATA_DIR,"Testing")


os.path.isdir(DATA_DIR) # assertion that we are pointed to correct folder
WEIGHTS_DIR = os.path.join("..","weights_and_config")
HERRING_WEIGHTS = os.path.join(WEIGHTS_DIR,"Herring")


images = glob.glob(os.path.join(TRAINING_DIR,"Batch 1","*.JPG")) + glob.glob(os.path.join(TRAINING_DIR,"Batch 1","*.jpg"))
annotations = glob.glob(os.path.join(TRAINING_DIR,"Batch 1 annotations","*.xml"))
print("{} images. Annotations: {}".format(len(images),len(annotations)))

144 images. Annotations: 108


## Scoring annotations against inference
**Requirements:** 2 folders: 1 with annotations (e.g. XML), 1 with images
* **Steps:**
  * 1) Run loop on images extracting predicted boxes from each image
  * 2) Scan filenames until box is properly extracted
  * 3) **Scoring Method:**
    * For each annotation find closest/any overlapping boxes (scoring the greater of number of either the annotated or predicted set distinctly)
    * Return average IOU for image
    * Return total number of correctly detected fish

In [11]:
idx = 1
print(annotations[idx])
tree = ET.parse(annotations[idx])
root = tree.getroot()

xmins = []
ymins = []
xmaxs = []
ymaxs = []
for o in root.iter("object"):
    if o.find("name").text == 'herring':
        xmins.append(o[4][0].text)
        ymins.append(o[4][1].text)
        xmaxs.append(o[4][2].text)
        ymaxs.append(o[4][3].text)
xmins,xmaxs,ymins,ymaxs

../data/Herring/Training/IRWA 2017 (Large+annotations)/Batch 1 annotations/2_2017-04-14_11-42-13_large.xml


(['56'], ['155'], ['135'], ['171'])

In [36]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	return interArea / float(boxAArea + boxBArea - interArea)



In [ ]:
# Scoring for one image
for box in boxes:
    boxA = 
    bb_intersection_over_union(boxA=,boxB=)

In [23]:
annotations[idx]

'../data/Herring/Training/IRWA 2017 (Large+annotations)/Batch 1 annotations/2_2017-04-13_12-26-28_large.xml'

## Evaluation

In [21]:

## Initialize the parameters
# Confidence threshold
conf_threshold = 0.5
# Non-maximum suppression threshold (maximum bounding box)
nms_threshold = 0.05
input_width = 416 # Width of network's input image
input_height = 416  # Height of network's input image

# Load class name
classes = "Herring"
# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = os.path.join(HERRING_WEIGHTS,"herring.cfg")
modelWeights = os.path.join(HERRING_WEIGHTS,"herring_final.weights") 
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)

for herring_image in images:

    if not annotations[idx].split('/')[-1].split('.')[0] in herring_image:
        continue

    print(herring_image)
    frame = cv2.imread(herring_image)
    
    # # Get frame from the video
    # hasFrame, frame = cap.read()

    input_width,input_height,_ = frame.shape
    input_width = input_height = min(input_width,input_height)
    input_width = 416  # Width of network's input image
    input_height = 416  # Height of network's input image

    print(input_height,input_width)
    # Create a 4D blob from a frame.
    blob = cv2.dnn.blobFromImage(
        frame, 1 / 255, (input_width, input_height), [0, 0, 0], 1, crop=False
    )
    
    # Sets the input to the network
    net.setInput(blob)

    # Runs the forward pass to get output of the output layers
    outs = net.forward(inference_utils.get_outputs_names(net=net))
    #break
    # Remove the bounding boxes with low confidence
    counts, boxes = inference_utils.postprocess(
        frame=frame,
        outs=outs,
        tracker=None,
        conf_threshold=conf_threshold,
        nms_threshold=nms_threshold,
        classes=classes,
    )

    
    if herring_image == [i for i in images if '2_2017-04-16_15-30-08_large' in i][0]:
        cv2.imshow(herring_image, frame)
        print(boxes)
        cv2.waitKey(0)
    print("Counts:",counts)
    print()
cv2.destroyAllWindows()

../data/Herring/Training/IRWA 2017 (Large+annotations)/Batch 1/2_2017-04-13_12-26-28_large.jpg
416 416
[0.37753522 0.6553458  0.2568322  0.20009485 0.6747363  0.6747363 ]
BOXES [[79, 133, 82, 48]] [0] [0]
counts 1
Counts: 1

